In [1]:
# transformer-decoder with no Seq2Seq component (autoregressive)

# no value embedding
# sine-cosine positional encoding on the hour, day, and month of timestamp
# modified transformer-encoder layer for masked self-attention

# conduct an architecture test similar to the one on the transformer-encoder in
# deep_transformer_model_for_tsf_XX.ipynb

In [2]:
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

In [4]:
print(tf.__version__)

2.4.1


In [5]:
# required for TFA MultiHeadAttention
import typing
import warnings

In [6]:
# MHA class from TensorFlow AddOns source
# it is compatible with TF 1.15 for CloudTPU usage

In [7]:
class MultiHeadAttention(tf.keras.layers.Layer):
    r"""MultiHead Attention layer.
    Defines the MultiHead Attention operation as described in
    [Attention Is All You Need](https://arxiv.org/abs/1706.03762) which takes
    in the tensors `query`, `key`, and `value`, and returns the dot-product attention
    between them:
    >>> mha = MultiHeadAttention(head_size=128, num_heads=12)
    >>> query = np.random.rand(3, 5, 4) # (batch_size, query_elements, query_depth)
    >>> key = np.random.rand(3, 6, 5) # (batch_size, key_elements, key_depth)
    >>> value = np.random.rand(3, 6, 6) # (batch_size, key_elements, value_depth)
    >>> attention = mha([query, key, value]) # (batch_size, query_elements, value_depth)
    >>> attention.shape
    TensorShape([3, 5, 6])
    If `value` is not given then internally `value = key` will be used:
    >>> mha = MultiHeadAttention(head_size=128, num_heads=12)
    >>> query = np.random.rand(3, 5, 5) # (batch_size, query_elements, query_depth)
    >>> key = np.random.rand(3, 6, 10) # (batch_size, key_elements, key_depth)
    >>> attention = mha([query, key]) # (batch_size, query_elements, key_depth)
    >>> attention.shape
    TensorShape([3, 5, 10])
    Args:
        head_size: int, dimensionality of the `query`, `key` and `value` tensors
            after the linear transformation.
        num_heads: int, number of attention heads.
        output_size: int, dimensionality of the output space, if `None` then the
            input dimension of `value` or `key` will be used,
            default `None`.
        dropout: float, `rate` parameter for the dropout layer that is
            applied to attention after softmax,
        default `0`.
        use_projection_bias: bool, whether to use a bias term after the linear
            output projection.
        return_attn_coef: bool, if `True`, return the attention coefficients as
            an additional output argument.
        kernel_initializer: initializer, initializer for the kernel weights.
        kernel_regularizer: regularizer, regularizer for the kernel weights.
        kernel_constraint: constraint, constraint for the kernel weights.
        bias_initializer: initializer, initializer for the bias weights.
        bias_regularizer: regularizer, regularizer for the bias weights.
        bias_constraint: constraint, constraint for the bias weights.
    Call Args:
        inputs:  List of `[query, key, value]` where
            * `query`: Tensor of shape `(..., query_elements, query_depth)`
            * `key`: `Tensor of shape '(..., key_elements, key_depth)`
            * `value`: Tensor of shape `(..., key_elements, value_depth)`, optional, if not given `key` will be used.
        mask: a binary Tensor of shape `[batch_size?, num_heads?, query_elements, key_elements]`
        which specifies which query elements can attend to which key elements,
        `1` indicates attention and `0` indicates no attention.
    Output shape:
        * `(..., query_elements, output_size)` if `output_size` is given, else
        * `(..., query_elements, value_depth)` if `value` is given, else
        * `(..., query_elements, key_depth)`
    """

    def __init__(
        self,
        head_size: int,
        num_heads: int,
        output_size: int = None,
        dropout: float = 0.0,
        use_projection_bias: bool = True,
        return_attn_coef: bool = False,
        kernel_initializer: typing.Union[str, typing.Callable] = "glorot_uniform",
        kernel_regularizer: typing.Union[str, typing.Callable] = None,
        kernel_constraint: typing.Union[str, typing.Callable] = None,
        bias_initializer: typing.Union[str, typing.Callable] = "zeros",
        bias_regularizer: typing.Union[str, typing.Callable] = None,
        bias_constraint: typing.Union[str, typing.Callable] = None,
        **kwargs,
    ):
        warnings.warn(
            "`MultiHeadAttention` will be deprecated in Addons 0.13. "
            "Please use `tf.keras.layers.MultiHeadAttention` instead.",
            DeprecationWarning,
        )

        super().__init__(**kwargs)

        if output_size is not None and output_size < 1:
            raise ValueError("output_size must be a positive number")

        self.head_size = head_size
        self.num_heads = num_heads
        self.output_size = output_size
        self.use_projection_bias = use_projection_bias
        self.return_attn_coef = return_attn_coef

        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)

        self.dropout = tf.keras.layers.Dropout(dropout)
        self._dropout_rate = dropout

    def build(self, input_shape):

        num_query_features = input_shape[0][-1]
        num_key_features = input_shape[1][-1]
        num_value_features = (
            input_shape[2][-1] if len(input_shape) > 2 else num_key_features
        )
        output_size = (
            self.output_size if self.output_size is not None else num_value_features
        )

        self.query_kernel = self.add_weight(
            name="query_kernel",
            shape=[self.num_heads, num_query_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.key_kernel = self.add_weight(
            name="key_kernel",
            shape=[self.num_heads, num_key_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.value_kernel = self.add_weight(
            name="value_kernel",
            shape=[self.num_heads, num_value_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.projection_kernel = self.add_weight(
            name="projection_kernel",
            shape=[self.num_heads, self.head_size, output_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )

        if self.use_projection_bias:
            self.projection_bias = self.add_weight(
                name="projection_bias",
                shape=[output_size],
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.projection_bias = None

        super().build(input_shape)

    def call(self, inputs, training=None, mask=None):

        # einsum nomenclature
        # ------------------------
        # N = query elements
        # M = key/value elements
        # H = heads
        # I = input features
        # O = output features

        query = inputs[0]
        key = inputs[1]
        value = inputs[2] if len(inputs) > 2 else key

        # verify shapes
        if key.shape[-2] != value.shape[-2]:
            raise ValueError(
                "the number of elements in 'key' must be equal to the same as the number of elements in 'value'"
            )

        if mask is not None:
            if len(mask.shape) < 2:
                raise ValueError("'mask' must have atleast 2 dimensions")
            if query.shape[-2] != mask.shape[-2]:
                raise ValueError(
                    "mask's second to last dimension must be equal to the number of elements in 'query'"
                )
            if key.shape[-2] != mask.shape[-1]:
                raise ValueError(
                    "mask's last dimension must be equal to the number of elements in 'key'"
                )

        # Linear transformations
        query = tf.einsum("...NI , HIO -> ...NHO", query, self.query_kernel)
        key = tf.einsum("...MI , HIO -> ...MHO", key, self.key_kernel)
        value = tf.einsum("...MI , HIO -> ...MHO", value, self.value_kernel)

        # Scale dot-product, doing the division to either query or key
        # instead of their product saves some computation
        depth = tf.constant(self.head_size, dtype=query.dtype)
        query /= tf.sqrt(depth)

        # Calculate dot product attention
        logits = tf.einsum("...NHO,...MHO->...HNM", query, key)

        # apply mask
        if mask is not None:
            mask = tf.cast(mask, tf.float32)

            # possibly expand on the head dimension so broadcasting works
            if len(mask.shape) != len(logits.shape):
                mask = tf.expand_dims(mask, -3)

            logits += -10e9 * (1.0 - mask)

        attn_coef = tf.nn.softmax(logits)

        # attention dropout
        attn_coef_dropout = self.dropout(attn_coef, training=training)

        # attention * value
        multihead_output = tf.einsum("...HNM,...MHI->...NHI", attn_coef_dropout, value)

        # Run the outputs through another linear projection layer. Recombining heads
        # is automatically done.
        output = tf.einsum(
            "...NHI,HIO->...NO", multihead_output, self.projection_kernel
        )

        if self.projection_bias is not None:
            output += self.projection_bias

        if self.return_attn_coef:
            return output, attn_coef
        else:
            return output

    def compute_output_shape(self, input_shape):
        num_value_features = (
            input_shape[2][-1] if len(input_shape) > 2 else input_shape[1][-1]
        )
        output_size = (
            self.output_size if self.output_size is not None else num_value_features
        )

        output_shape = input_shape[0][:-1] + (output_size,)

        if self.return_attn_coef:
            num_query_elements = input_shape[0][-2]
            num_key_elements = input_shape[1][-2]
            attn_coef_shape = input_shape[0][:-2] + (
                self.num_heads,
                num_query_elements,
                num_key_elements,
            )

            return output_shape, attn_coef_shape
        else:
            return output_shape

    def get_config(self):
        config = super().get_config()

        config.update(
            head_size=self.head_size,
            num_heads=self.num_heads,
            output_size=self.output_size,
            dropout=self._dropout_rate,
            use_projection_bias=self.use_projection_bias,
            return_attn_coef=self.return_attn_coef,
            kernel_initializer=tf.keras.initializers.serialize(self.kernel_initializer),
            kernel_regularizer=tf.keras.regularizers.serialize(self.kernel_regularizer),
            kernel_constraint=tf.keras.constraints.serialize(self.kernel_constraint),
            bias_initializer=tf.keras.initializers.serialize(self.bias_initializer),
            bias_regularizer=tf.keras.regularizers.serialize(self.bias_regularizer),
            bias_constraint=tf.keras.constraints.serialize(self.bias_constraint),
        )

        return config

In [8]:
# generate triangular mask for self-attention
# as a TensorFlow tensor
d = 3
tf.convert_to_tensor(np.tril(np.ones([d, d]), 0), dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[1., 0., 0.],
       [1., 1., 0.],
       [1., 1., 1.]], dtype=float32)>

In [9]:
# the autoregressive version of the transformer-decoder does not use the Seq2Seq intermediate layer
# as there is no transformer-encoder component that sends encoding hidden states, therefore
# having only a self-attention layer and position-wise feed-forward layer,
# the autoregressive transformer-decoder is, in fact, a transformer-encoder

# the only important modification is the masked self-attention layer

# masked self-attention layer seems to be already implemented in
# MHA module from TensorFlow AddOns

In [10]:
# base transformer encoder layer from # https://keras.io/examples/nlp/text_classification_with_transformer/
# modified to include masked self attention

# pass number of timesteps as an argument for the encoder layer
# ToDo: get the number of timesteps from the input shape
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, n_timesteps, embed_dim, num_heads, ff_dim, dropout=0.1):
        super(EncoderLayer, self).__init__()
        # multi-head attention initialization
        self.attention_layer = MultiHeadAttention(head_size=embed_dim, num_heads=num_heads)
        self.ff_layer = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"),
             tf.keras.layers.Dense(embed_dim)]
        )
        self.add_norm_layer_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.add_norm_layer_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout_1 = tf.keras.layers.Dropout(dropout)
        self.dropout_2 = tf.keras.layers.Dropout(dropout)
        # mask for self-attention
        self.mask = tf.convert_to_tensor(np.tril(np.ones([n_timesteps, n_timesteps]), 0), dtype=tf.float32)

        
    def call(self, inputs, training):
        # mask for self-attention is passed to MHA on call
        attention_output = self.attention_layer([inputs, inputs], mask=self.mask)
        attention_output = self.dropout_1(attention_output, training=training)
        input_to_ffn = self.add_norm_layer_1(inputs + attention_output)
        ffn_output = self.ff_layer(input_to_ffn)
        ffn_output = self.dropout_2(ffn_output, training=training)
        return self.add_norm_layer_2(input_to_ffn + ffn_output)

In [11]:
# get the active power time series as main data source
! ls -l /home/developer/gcp/cbidmltsf/timeseries

total 8
drwxrwxr-x 2 developer developer 4096 feb 11 13:08 CPE04115_H_kw_20201021084001
drwxrwxr-x 2 developer developer 4096 feb 16 13:17 CPE04115_H_kw_20201021084001_csv


In [12]:
! ls -l /home/developer/gcp/cbidmltsf/timeseries/CPE04115_H_kw_20201021084001

total 364
-rw-rw-r-- 1 developer developer    621 oct 21  2020 scaler.save
-rw-rw-r-- 1 developer developer    218 oct 21  2020 ts.json
-rw-rw-r-- 1 developer developer 362977 oct 21  2020 ts.pkl


In [13]:
ts = pd.read_pickle("/home/developer/gcp/cbidmltsf/timeseries/CPE04115_H_kw_20201021084001/ts.pkl")

In [14]:
ts.head()

,kw_scaled
timestamp,
2016-01-01 00:00:00,0.274317
2016-01-01 01:00:00,0.217363
2016-01-01 02:00:00,0.168545
2016-01-01 03:00:00,0.122996
2016-01-01 04:00:00,0.080440


In [15]:
# build a training dataset
# features: m consecutive lectures with their timestamps
# target: m consecutive lectures (lectures in features, shifted by 1 to the future)

In [16]:
# length of time series
ts['kw_scaled'].count()

22629

In [17]:
# length of input sequence
m = 48

In [18]:
# prepare sine-cosine positional encoding for the input sequence
hours_in_day = 24
days_in_month = 30
months_in_year = 12

In [19]:
# get a collection of examples for training
# use a range on the time series index
train_range = np.arange(0, 1000)

features_list = list()
targets_list = list()

for start in train_range:
    end = start + m
    values = np.expand_dims(ts[start:end]['kw_scaled'].values, axis=1)
    target_values = np.expand_dims(ts[1+start:1+end]['kw_scaled'].values, axis=1)
    
    timestamps_hour = ts[start:end].index.hour
    timestamps_day = ts[start:end].index.day
    timestamps_month = ts[start:end].index.month
    
    sin_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    cos_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    sin_day = np.expand_dims(np.sin(2*np.pi*timestamps_day/days_in_month), axis=1)
    cos_day = np.expand_dims(np.cos(2*np.pi*timestamps_day/days_in_month), axis=1)
    sin_month = np.expand_dims(np.sin(2*np.pi*timestamps_month/months_in_year), axis=1)
    cos_month = np.expand_dims(np.cos(2*np.pi*timestamps_month/months_in_year), axis=1)
    
    feature_row = np.concatenate((values, sin_hour, cos_hour, sin_day, cos_day, sin_month, cos_month), axis=1)    

    features_list.append(feature_row)
    targets_list.append(target_values)

x_train = np.array(features_list)
y_train = np.array(targets_list)

In [20]:
x_train.shape, y_train.shape

((1000, 48, 7), (1000, 48, 1))

In [21]:
x_train[0, :, 0]

array([0.27431688, 0.21736328, 0.16854513, 0.12299635, 0.08044036,
       0.04925277, 0.06771694, 0.04966028, 0.02315827, 0.0611418 ,
       0.16809889, 0.23126119, 0.28965889, 0.31682717, 0.34027286,
       0.37148524, 0.3697847 , 0.38247015, 0.44962929, 0.56348924,
       0.53345987, 0.51550237, 0.43560898, 0.32928794, 0.23602656,
       0.14696731, 0.09737444, 0.04251571, 0.04444092, 0.06843589,
       0.12769277, 0.15101761, 0.1872332 , 0.24428829, 0.30788212,
       0.37800538, 0.41353456, 0.43940206, 0.43413931, 0.40126436,
       0.37682159, 0.41638402, 0.47179745, 0.59373552, 0.59430882,
       0.59690262, 0.53751327, 0.3898944 ])

In [22]:
y_train[0, :, 0]

array([0.21736328, 0.16854513, 0.12299635, 0.08044036, 0.04925277,
       0.06771694, 0.04966028, 0.02315827, 0.0611418 , 0.16809889,
       0.23126119, 0.28965889, 0.31682717, 0.34027286, 0.37148524,
       0.3697847 , 0.38247015, 0.44962929, 0.56348924, 0.53345987,
       0.51550237, 0.43560898, 0.32928794, 0.23602656, 0.14696731,
       0.09737444, 0.04251571, 0.04444092, 0.06843589, 0.12769277,
       0.15101761, 0.1872332 , 0.24428829, 0.30788212, 0.37800538,
       0.41353456, 0.43940206, 0.43413931, 0.40126436, 0.37682159,
       0.41638402, 0.47179745, 0.59373552, 0.59430882, 0.59690262,
       0.53751327, 0.3898944 , 0.27423011])

In [23]:
# training with teacher forcing
# pass only true values

In [24]:
# get a collection of examples for evaluation
# use a range on the time series index
eval_range = np.arange(1100, 1300)

features_list = list()
targets_list = list()

for start in eval_range:
    end = start + m
    values = np.expand_dims(ts[start:end]['kw_scaled'].values, axis=1)
    target_values = np.expand_dims(ts[1+start:1+end]['kw_scaled'].values, axis=1)
    
    timestamps_hour = ts[start:end].index.hour
    timestamps_day = ts[start:end].index.day
    timestamps_month = ts[start:end].index.month
    
    sin_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    cos_hour = np.expand_dims(np.sin(2*np.pi*timestamps_hour/hours_in_day), axis=1)
    sin_day = np.expand_dims(np.sin(2*np.pi*timestamps_day/days_in_month), axis=1)
    cos_day = np.expand_dims(np.cos(2*np.pi*timestamps_day/days_in_month), axis=1)
    sin_month = np.expand_dims(np.sin(2*np.pi*timestamps_month/months_in_year), axis=1)
    cos_month = np.expand_dims(np.cos(2*np.pi*timestamps_month/months_in_year), axis=1)
    
    feature_row = np.concatenate((values, sin_hour, cos_hour, sin_day, cos_day, sin_month, cos_month), axis=1)    

    features_list.append(feature_row)
    targets_list.append(target_values)

x_eval = np.array(features_list)
y_eval = np.array(targets_list)

In [25]:
x_eval.shape, y_eval.shape

((200, 48, 7), (200, 48, 1))

In [26]:
x_eval[0, :, 0]

array([0.61518706, 0.57427117, 0.4674117 , 0.34486934, 0.2245218 ,
       0.15264377, 0.10878081, 0.07731354, 0.05909031, 0.05338674,
       0.14980825, 0.19518737, 0.20623891, 0.21277067, 0.29595125,
       0.31055726, 0.33621482, 0.36981647, 0.37839197, 0.38991145,
       0.39248666, 0.40448492, 0.42273217, 0.55039318, 0.61994778,
       0.56638596, 0.45977285, 0.31207109, 0.19304601, 0.11578748,
       0.08181241, 0.07105061, 0.05574425, 0.07363279, 0.17405657,
       0.23519682, 0.27807665, 0.33219551, 0.399436  , 0.41929778,
       0.47372576, 0.51229809, 0.51688914, 0.5222681 , 0.51115691,
       0.52278717, 0.55290486, 0.69725978])

In [30]:
y_eval[0, :, 0]

array([0.57427117, 0.4674117 , 0.34486934, 0.2245218 , 0.15264377,
       0.10878081, 0.07731354, 0.05909031, 0.05338674, 0.14980825,
       0.19518737, 0.20623891, 0.21277067, 0.29595125, 0.31055726,
       0.33621482, 0.36981647, 0.37839197, 0.38991145, 0.39248666,
       0.40448492, 0.42273217, 0.55039318, 0.61994778, 0.56638596,
       0.45977285, 0.31207109, 0.19304601, 0.11578748, 0.08181241,
       0.07105061, 0.05574425, 0.07363279, 0.17405657, 0.23519682,
       0.27807665, 0.33219551, 0.399436  , 0.41929778, 0.47372576,
       0.51229809, 0.51688914, 0.5222681 , 0.51115691, 0.52278717,
       0.55290486, 0.69725978, 0.68099739])

In [31]:
# architecture details according to the Klingenbrunn experiment
# (including notes to further modifications on the basic autoregressive model)

In [32]:
# number of timesteps is the length of the input sequence,
# is the embedding dimension from SLDB
num_timesteps = m

In [33]:
# number of features is the active load value (main feature)
# plus the six components of the sine-cosine positional encoding on hour, day, month

# important: there is no value embedding, therefore d_model is very low
d_model = 7

# ToDo: use value embedding to a high-dimensional space and compare results
# ToDo: use a different positional encoding system and compare results

In [34]:
# as long as there is no value embedding, neither convolutional nor dense layers are required

In [35]:
# input layer for Keras functional
input_layer = tf.keras.layers.Input(shape=(num_timesteps, d_model))
input_layer

<KerasTensor: shape=(None, 48, 7) dtype=float32 (created by layer 'input_1')>

In [36]:
input_to_transformer_block = input_layer
input_to_transformer_block

<KerasTensor: shape=(None, 48, 7) dtype=float32 (created by layer 'input_1')>

In [55]:
num_heads = 4
ff_dim = 256
dropout = 0.2

In [56]:
encoder_layer_1 = EncoderLayer(n_timesteps=num_timesteps,
                               embed_dim=d_model,
                               num_heads=num_heads,
                               ff_dim=ff_dim,
                               dropout=dropout)

/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: `MultiHeadAttention` will be deprecated in Addons 0.13. Please use `tf.keras.layers.MultiHeadAttention` instead.


In [64]:
encoder_layer_2 = EncoderLayer(n_timesteps=num_timesteps,
                               embed_dim=d_model,
                               num_heads=num_heads,
                               ff_dim=ff_dim,
                               dropout=dropout)

/home/developer/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:74: DeprecationWarning: `MultiHeadAttention` will be deprecated in Addons 0.13. Please use `tf.keras.layers.MultiHeadAttention` instead.


In [57]:
output_from_encoder_1 = encoder_layer_1(input_to_transformer_block)
output_from_encoder_1

<KerasTensor: shape=(None, 48, 7) dtype=float32 (created by layer 'encoder_layer_2')>

In [65]:
output_from_encoder_2 = encoder_layer_2(output_from_encoder_1)
output_from_encoder_2

<KerasTensor: shape=(None, 48, 7) dtype=float32 (created by layer 'encoder_layer_3')>

In [66]:
# Klingenbrunn uses a linear layer to decode the output_from_encoder
# from (?, num_timesteps, num_features) to (?, num_timesteps, 1)

# the equivalent operation in TensorFlow is a TimeDistributed Dense layer to 1

In [67]:
units_in_first_dense = 1
first_dense = tf.keras.layers.Dense(units_in_first_dense, activation="sigmoid")

In [68]:
distributed_first_dense = tf.keras.layers.TimeDistributed(first_dense)(output_from_encoder_2)
distributed_first_dense

<KerasTensor: shape=(None, 48, 1) dtype=float32 (created by layer 'time_distributed_3')>

In [72]:
model = tf.keras.Model(inputs=input_layer, outputs=distributed_first_dense)

In [73]:
model.compile("adam", "mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])

In [74]:
history = model.fit(
    x_train, y_train, batch_size=32, epochs=100, validation_data=(x_eval, y_eval)
)

Epoch 1/500
32/32 [==============================] - 3s 34ms/step - loss: 0.0026 - root_mean_squared_error: 0.0506 - val_loss: 0.0035 - val_root_mean_squared_error: 0.0596
Epoch 2/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0014 - root_mean_squared_error: 0.0369 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0620
Epoch 3/500
32/32 [==============================] - 1s 22ms/step - loss: 0.0013 - root_mean_squared_error: 0.0360 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0624
Epoch 4/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0013 - root_mean_squared_error: 0.0356 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0636
Epoch 5/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0013 - root_mean_squared_error: 0.0360 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0618
Epoch 6/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0013 - root_mean_squared_error: 0.0362 - val_loss: 0.0037 - val_

32/32 [==============================] - 1s 23ms/step - loss: 0.0012 - root_mean_squared_error: 0.0342 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0627
Epoch 49/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0011 - root_mean_squared_error: 0.0330 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0633
Epoch 50/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0011 - root_mean_squared_error: 0.0332 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0612
Epoch 51/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0013 - root_mean_squared_error: 0.0360 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0625
Epoch 52/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0011 - root_mean_squared_error: 0.0336 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0653
Epoch 53/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0011 - root_mean_squared_error: 0.0338 - val_loss: 0.0040 - val_root_me

32/32 [==============================] - 1s 23ms/step - loss: 0.0010 - root_mean_squared_error: 0.0322 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0629
Epoch 96/500
32/32 [==============================] - 1s 22ms/step - loss: 0.0010 - root_mean_squared_error: 0.0319 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0624
Epoch 97/500
32/32 [==============================] - 1s 23ms/step - loss: 9.9777e-04 - root_mean_squared_error: 0.0316 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0601
Epoch 98/500
32/32 [==============================] - 1s 22ms/step - loss: 0.0010 - root_mean_squared_error: 0.0319 - val_loss: 0.0035 - val_root_mean_squared_error: 0.0593
Epoch 99/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0010 - root_mean_squared_error: 0.0320 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0614
Epoch 100/500
32/32 [==============================] - 1s 23ms/step - loss: 0.0010 - root_mean_squared_error: 0.0321 - val_loss: 0.0041 - val_ro

Epoch 142/500
32/32 [==============================] - 1s 23ms/step - loss: 9.5954e-04 - root_mean_squared_error: 0.0310 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0604
Epoch 143/500
32/32 [==============================] - 1s 22ms/step - loss: 9.3915e-04 - root_mean_squared_error: 0.0306 - val_loss: 0.0044 - val_root_mean_squared_error: 0.0664
Epoch 144/500
32/32 [==============================] - 1s 26ms/step - loss: 9.7035e-04 - root_mean_squared_error: 0.0311 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0610
Epoch 145/500
32/32 [==============================] - 1s 23ms/step - loss: 9.7539e-04 - root_mean_squared_error: 0.0312 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0600
Epoch 146/500
32/32 [==============================] - 1s 23ms/step - loss: 9.3141e-04 - root_mean_squared_error: 0.0305 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0607
Epoch 147/500
32/32 [==============================] - 1s 23ms/step - loss: 9.3414e-04 - root_mean_squared_err

32/32 [==============================] - 1s 22ms/step - loss: 8.9477e-04 - root_mean_squared_error: 0.0299 - val_loss: 0.0032 - val_root_mean_squared_error: 0.0569
Epoch 189/500
32/32 [==============================] - 1s 22ms/step - loss: 9.1321e-04 - root_mean_squared_error: 0.0302 - val_loss: 0.0035 - val_root_mean_squared_error: 0.0595
Epoch 190/500
32/32 [==============================] - 1s 23ms/step - loss: 8.8652e-04 - root_mean_squared_error: 0.0298 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0603
Epoch 191/500
32/32 [==============================] - 1s 23ms/step - loss: 8.6467e-04 - root_mean_squared_error: 0.0294 - val_loss: 0.0035 - val_root_mean_squared_error: 0.0590
Epoch 192/500
32/32 [==============================] - 1s 23ms/step - loss: 8.9297e-04 - root_mean_squared_error: 0.0299 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0656
Epoch 193/500
32/32 [==============================] - 1s 22ms/step - loss: 9.6476e-04 - root_mean_squared_error: 0.0311 - v

...

32/32 [==============================] - 1s 22ms/step - loss: 8.3622e-04 - root_mean_squared_error: 0.0289 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0619
Epoch 281/500
32/32 [==============================] - 1s 23ms/step - loss: 8.3325e-04 - root_mean_squared_error: 0.0289 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0599
Epoch 282/500
32/32 [==============================] - 1s 23ms/step - loss: 8.4290e-04 - root_mean_squared_error: 0.0290 - val_loss: 0.0032 - val_root_mean_squared_error: 0.0569
Epoch 283/500
32/32 [==============================] - 1s 23ms/step - loss: 8.1519e-04 - root_mean_squared_error: 0.0285 - val_loss: 0.0048 - val_root_mean_squared_error: 0.0689
Epoch 284/500
32/32 [==============================] - 1s 22ms/step - loss: 8.4472e-04 - root_mean_squared_error: 0.0291 - val_loss: 0.0039 - val_root_mean_squared_error: 0.0621
Epoch 285/500
32/32 [==============================] - 1s 22ms/step - loss: 8.4610e-04 - root_mean_squared_error: 0.0291 - v

32/32 [==============================] - 1s 23ms/step - loss: 8.0523e-04 - root_mean_squared_error: 0.0284 - val_loss: 0.0041 - val_root_mean_squared_error: 0.0643
Epoch 327/500
32/32 [==============================] - 1s 23ms/step - loss: 8.4372e-04 - root_mean_squared_error: 0.0290 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0652
Epoch 328/500
32/32 [==============================] - 1s 23ms/step - loss: 7.7396e-04 - root_mean_squared_error: 0.0278 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0603
Epoch 329/500
32/32 [==============================] - 1s 23ms/step - loss: 8.1155e-04 - root_mean_squared_error: 0.0285 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0659
Epoch 330/500
32/32 [==============================] - 1s 22ms/step - loss: 8.0373e-04 - root_mean_squared_error: 0.0283 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0655
Epoch 331/500
32/32 [==============================] - 1s 23ms/step - loss: 8.1588e-04 - root_mean_squared_error: 0.0286 - v

32/32 [==============================] - 1s 23ms/step - loss: 7.6077e-04 - root_mean_squared_error: 0.0276 - val_loss: 0.0036 - val_root_mean_squared_error: 0.0603
Epoch 373/500
32/32 [==============================] - 1s 23ms/step - loss: 7.8064e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0034 - val_root_mean_squared_error: 0.0582
Epoch 374/500
32/32 [==============================] - 1s 23ms/step - loss: 9.0039e-04 - root_mean_squared_error: 0.0300 - val_loss: 0.0044 - val_root_mean_squared_error: 0.0666
Epoch 375/500
32/32 [==============================] - 1s 23ms/step - loss: 7.6076e-04 - root_mean_squared_error: 0.0276 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0608
Epoch 376/500
32/32 [==============================] - 1s 22ms/step - loss: 7.7348e-04 - root_mean_squared_error: 0.0278 - val_loss: 0.0053 - val_root_mean_squared_error: 0.0731
Epoch 377/500
32/32 [==============================] - 1s 22ms/step - loss: 7.6748e-04 - root_mean_squared_error: 0.0277 - v

32/32 [==============================] - 1s 23ms/step - loss: 8.2526e-04 - root_mean_squared_error: 0.0287 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0629
Epoch 419/500
32/32 [==============================] - 1s 22ms/step - loss: 7.8118e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0037 - val_root_mean_squared_error: 0.0606
Epoch 420/500
32/32 [==============================] - 1s 23ms/step - loss: 7.6898e-04 - root_mean_squared_error: 0.0277 - val_loss: 0.0043 - val_root_mean_squared_error: 0.0652
Epoch 421/500
32/32 [==============================] - 1s 23ms/step - loss: 7.7902e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0038 - val_root_mean_squared_error: 0.0616
Epoch 422/500
32/32 [==============================] - 1s 23ms/step - loss: 7.8238e-04 - root_mean_squared_error: 0.0280 - val_loss: 0.0050 - val_root_mean_squared_error: 0.0706
Epoch 423/500
32/32 [==============================] - 1s 23ms/step - loss: 7.4147e-04 - root_mean_squared_error: 0.0272 - v

32/32 [==============================] - 1s 23ms/step - loss: 7.1280e-04 - root_mean_squared_error: 0.0267 - val_loss: 0.0052 - val_root_mean_squared_error: 0.0722
Epoch 465/500
32/32 [==============================] - 1s 23ms/step - loss: 8.0129e-04 - root_mean_squared_error: 0.0283 - val_loss: 0.0040 - val_root_mean_squared_error: 0.0632
Epoch 466/500
32/32 [==============================] - 1s 23ms/step - loss: 7.7265e-04 - root_mean_squared_error: 0.0278 - val_loss: 0.0044 - val_root_mean_squared_error: 0.0666
Epoch 467/500
32/32 [==============================] - 1s 23ms/step - loss: 7.7133e-04 - root_mean_squared_error: 0.0278 - val_loss: 0.0042 - val_root_mean_squared_error: 0.0646
Epoch 468/500
32/32 [==============================] - 1s 23ms/step - loss: 7.8111e-04 - root_mean_squared_error: 0.0279 - val_loss: 0.0046 - val_root_mean_squared_error: 0.0675
Epoch 469/500
32/32 [==============================] - 1s 23ms/step - loss: 7.7020e-04 - root_mean_squared_error: 0.0278 - v

In [75]:
# attempt a Keras model prediction based on history

In [100]:
# get the row from evaluation dataset as feature
x = tf.expand_dims(x_eval[-1], axis=0)
x.shape

TensorShape([1, 48, 7])

In [101]:
prediction = model.predict(
    x, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False
)

In [102]:
prediction.shape

(1, 48, 1)

In [103]:
tf.squeeze(prediction)

<tf.Tensor: shape=(48,), dtype=float32, numpy=
array([0.05151626, 0.0556708 , 0.08318445, 0.21065259, 0.26449195,
       0.29920423, 0.35440305, 0.4246517 , 0.4375494 , 0.47477224,
       0.47573763, 0.5083699 , 0.547621  , 0.5605179 , 0.5864214 ,
       0.5884714 , 0.6397058 , 0.62527925, 0.57474184, 0.38857785,
       0.20128909, 0.13423377, 0.06665537, 0.05359823, 0.04958963,
       0.05151397, 0.07779732, 0.16744536, 0.27258268, 0.26778847,
       0.29126436, 0.39697954, 0.415762  , 0.45612934, 0.42826465,
       0.5056358 , 0.5836543 , 0.5700174 , 0.5550365 , 0.5969828 ,
       0.65022904, 0.6379078 , 0.5966716 , 0.41843218, 0.2176767 ,
       0.14470291, 0.07474747, 0.05372745], dtype=float32)>

In [104]:
y_eval[-1][:, 0]

array([0.05694353, 0.08109268, 0.1868071 , 0.2268336 , 0.27008607,
       0.34029998, 0.41255685, 0.46587463, 0.52656089, 0.53039813,
       0.542525  , 0.55296916, 0.55264532, 0.56996134, 0.56719632,
       0.70061591, 0.69560572, 0.62580088, 0.5194047 , 0.3447051 ,
       0.22073026, 0.1225563 , 0.09421663, 0.07533178, 0.03961047,
       0.05124383, 0.16601331, 0.24328269, 0.27739954, 0.32324039,
       0.39474112, 0.44984002, 0.49953826, 0.50764815, 0.5444812 ,
       0.5982034 , 0.55595497, 0.51901191, 0.5598511 , 0.69131216,
       0.69535239, 0.64015355, 0.533456  , 0.38168535, 0.25074026,
       0.15781743, 0.10555095, 0.09946311])